Overall
https://towardsdatascience.com/building-an-employee-churn-model-in-python-to-develop-a-strategic-retention-plan-57d5bd882c2d
- I saw only one hot encoding and no label encoding. The article stated we need both
- No MinMaxScaler, the article stated that there will be better results
- There could be a bigger problem with Train data, check my remarks
- Some code changes were applied
- at the end not everything was understood in regard to NN, as too many new and different functions were used.
- The NN accuracy is sometimes very random. Some learnings are getting ~15% accuracy some over ~80%. Looks loke there are two minimums. Maybe check something like gradient descent? Maybe its already used, then other minimum function.

Overall:
The most interesting regression was the Random Forest. Where you can get a ranking of most influantial things why people leave the company. This was very insigtfull. The NN can only predict how likely will someone will leave. NN is as always a black box algorithm. Maybe here just to think if NN is a good algorithm to have for this problem.
Untill now I have the feeling NN is only good if you dont understand anything and there some kind of totaly unstructured data. Here the data is very much structure and understandable.


In [74]:
# Colab only
#from google.colab import drive
#drive.mount('/content/gdrive')

Packages for Virtual Envirenment

In [3]:
# Packages for Virtual Environment
#! pip install tensorflow
#! pip install pandas
#! pip install sklearn
#! pip install xlrd

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import os 
import json
import codecs

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import feature_column
from tensorflow import keras

from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

This should not be neccassary please check next coding

In [51]:
#dtypes={
#    'Age':                         'int64',
#    'Attrition':                   'category',
#    'BusinessTravel':              'category',
#    'DailyRate':                   'int64',
#    'Department':                  'category',
#    'DistanceFromHome':            'int64',
#    'Education':                   'int64',
#    'EducationField':              'category',
#    'EmployeeCount':               'int64',
#    'EmployeeNumber':              'int64',
#    'EnvironmentSatisfaction':     'int64',
#    'Gender':                      'category',
#    'HourlyRate':                  'int64',
#    'JobInvolvement':              'int64',
#    'JobLevel':                    'int64',
#    'JobRole':                     'category',
#    'JobSatisfaction':             'int64',
#    'MaritalStatus':               'category',
#    'MonthlyIncome':               'int64',
#    'MonthlyRate':                 'int64',
#    'NumCompaniesWorked':          'int64',
#    'Over18':                      'category',
#    'OverTime':                    'category',
#    'PercentSalaryHike':           'int64',
#    'PerformanceRating':           'int64',
#    'RelationshipSatisfaction':    'int64',
#    'StandardHours':               'int64',
#    'StockOptionLevel':            'int64',
#    'TotalWorkingYears':           'int64',
#    'TrainingTimesLastYear':       'int64',
#    'WorkLifeBalance':             'int64',
#    'YearsAtCompany':              'int64',
#    'YearsInCurrentRole':          'int64',
#    'YearsSinceLastPromotion':     'int64',
#    'YearsWithCurrManager':        'int64',}

In [52]:
# For Colab
# df = pd.read_excel('/content/gdrive/My Drive/WA_Fn-UseC_-HR-Employee-Attrition.xlsx', sheet_name=0,dtype=dtypes)

# Local Data
df = pd.read_excel('WA_Fn-UseC_-HR-Employee-Attrition.xlsx', sheet_name=0,dtype=dtypes)

Here you can check for non int64 values for hot encoding. easier solution follows.

In [53]:
#dict = df.dtypes.to_dict()

#for key, value in dict.items():
#    print("{}-{}".format(key, value))


get_dummies is not categorical encoding but one hot encoding. Comment is not correct.
cont not used and not needed
cont.append not needed

In [54]:
## preprocessing : categorical encoding
#df['Attrition']=df.Attrition.eq('Yes').mul(1) # change target from Yes/no to 1/0
#cont=[]
#cat=[]
#for key, value in dtypes.items():
#    if key!='Attrition':
#        if value == "int64":
#            cont.append(key)
#        else:
#            cat.append(key)
#df = pd.get_dummies(df, columns=cat)
#df.head()

In [55]:
# preprocessing : categorical encoding
df['Attrition']=df.Attrition.eq('Yes').mul(1) # change target from Yes/no to 1/0
df = pd.get_dummies(df, columns=df.select_dtypes(['category']).columns)
# two rows for all the hot encoding. Isn't it great! :D
df.head()

,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1,1102,1,2,1,1,2,94,3,...,0,0,1,0,0,0,1,1,0,1
1,49,0,279,8,1,1,2,3,61,2,...,0,1,0,0,0,1,0,1,1,0
2,37,1,1373,2,2,1,4,4,92,2,...,0,0,0,0,0,0,1,1,0,1
3,33,0,1392,3,4,1,5,4,56,3,...,0,1,0,0,0,1,0,1,0,1
4,27,0,591,2,1,1,7,1,40,3,...,0,0,0,0,0,1,0,1,1,0


In [56]:
rename = ["Department_Human Resources",
          "Department_Research & Development" ,
"EducationField_Human Resources" ,
"EducationField_Life Sciences" ,
"EducationField_Technical Degree", 
"JobRole_Healthcare Representative" ,
"JobRole_Human Resources" ,
"JobRole_Laboratory Technician",
"JobRole_Manufacturing Director",
"JobRole_Research Director",
"JobRole_Research Scientist",
"JobRole_Sales Executive",
"JobRole_Sales Representative"]

for string in rename:
  df.rename(columns={ string : string.replace(" ", "_")}, inplace = True)

df.rename(columns={ "Department_Research_&_Development": "Department_ResearchDevelopment"}, inplace = True)
df.rename(columns={ "BusinessTravel_Non-Travel": "BusinessTravel_NonTravel"}, inplace = True)
df.head()








,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,JobRole_Research_Director,JobRole_Research_Scientist,JobRole_Sales_Executive,JobRole_Sales_Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1,1102,1,2,1,1,2,94,3,...,0,0,1,0,0,0,1,1,0,1
1,49,0,279,8,1,1,2,3,61,2,...,0,1,0,0,0,1,0,1,1,0
2,37,1,1373,2,2,1,4,4,92,2,...,0,0,0,0,0,0,1,1,0,1
3,33,0,1392,3,4,1,5,4,56,3,...,0,1,0,0,0,1,0,1,0,1
4,27,0,591,2,1,1,7,1,40,3,...,0,0,0,0,0,1,0,1,1,0


output takes too much space

In [57]:
#df.dtypes

Moved this coding

In [58]:
#cont = []
#cat = []

#for items in df.dtypes.iteritems(): 
#   if items[0]!='Attrition':
#    cont.append(items[0])

#print(cont)



train will be overwritten. This is strange. I am not very fammiliar with validation dataset. So not sure what to think about this.

In [59]:
train, test = train_test_split(df, test_size=0.2)
train1, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

1176 train examples
236 validation examples
294 test examples


dataframe = dataframe.copy() - is this correct? looks like it does not do anything

In [60]:

# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Attrition')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [61]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [62]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of Education:', feature_batch['Education'])
  print('A batch of Attrication:', label_batch )

Every feature: ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'BusinessTravel_NonTravel', 'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely', 'Department_Human_Resources', 'Department_ResearchDevelopment', 'Department_Sales', 'EducationField_Human_Resources', 'EducationField_Life_Sciences', 'EducationField_Marketing', 'EducationField_Medical', 'EducationField_Other', 'EducationField_Technical_Degree', 'Gender_Female', 'Gender_Male', 'JobRole_Healthcare_Representative', 'JobRole_Human_Resources', 'JobRole_Laboratory_Technician', '

In [63]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [64]:

# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [65]:
daysOpen = feature_column.numeric_column("Education")
demo(daysOpen)

[[3.]
 [3.]
 [1.]
 [4.]
 [4.]]


some coding moved here. I think there may be only one loop here.

In [66]:
feature_columns = []
cont = []
cat = []

for items in df.dtypes.iteritems(): 
   if items[0]!='Attrition':
    cont.append(items[0])

#dep_var = 'Attrition'

# numeric cols
for header in cont:
  if header != 'Attrition':
    feature_columns.append(feature_column.numeric_column(header))

In [67]:
print(feature_columns)

[NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='DailyRate', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='DistanceFromHome', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Education', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='EmployeeCount', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='EmployeeNumber', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='EnvironmentSatisfaction', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='HourlyRate', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='JobInvolvement', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='JobLevel', shape=(1,), default_v

In [68]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [69]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [70]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of Education:', feature_batch['Education'])
  print('A batch of Attration:', label_batch )

Every feature: ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'BusinessTravel_NonTravel', 'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely', 'Department_Human_Resources', 'Department_ResearchDevelopment', 'Department_Sales', 'EducationField_Human_Resources', 'EducationField_Life_Sciences', 'EducationField_Marketing', 'EducationField_Medical', 'EducationField_Other', 'EducationField_Technical_Degree', 'Gender_Female', 'Gender_Male', 'JobRole_Healthcare_Representative', 'JobRole_Human_Resources', 'JobRole_Laboratory_Technician', '

Very strange that the loss function is not moving at all. Its immidiately reaches the local minimum??

In [71]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(64, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(lr=1e-3),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'],
              run_eagerly=True)

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Epoch 1/5
37/37 [==============================] - 2s 51ms/step - loss: 12.8295 - accuracy: 0.1633 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
37/37 [==============================] - 2s 45ms/step - loss: 12.8439 - accuracy: 0.1633 - val_loss: 12.4583 - val_accuracy: 0.1822
Epoch 3/5
37/37 [==============================] - 2s 50ms/step - loss: 12.8050 - accuracy: 0.1633 - val_loss: 12.4583 - val_accuracy: 0.1822
Epoch 4/5
37/37 [==============================] - 2s 46ms/step - loss: 12.7921 - accuracy: 0.1633 - val_loss: 12.4583 - val_accuracy: 0.1822
Epoch 5/5
37/37 [==============================] - 2s 62ms/step - loss: 12.8309 - accuracy: 0.1633 - val_loss: 12.4583 - val_accuracy: 0.1822


In [72]:

loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

10/10 [==============================] - 0s 49ms/step - loss: 12.9694 - accuracy: 0.1531
Accuracy 0.15306123


In [73]:
i = 0
df2 =  df.drop(['Attrition'], axis=1)
x = np.array(df2.iloc[i])
x = x.reshape(-1,1)
y = x.tolist()
print(y)
file_path = "/content/gdrive/My Drive/hrtest.json"
#json.dump(y, codecs.open(file_path, 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True, indent=4)

[[41], [1102], [1], [2], [1], [1], [2], [94], [3], [2], [4], [5993], [19479], [8], [11], [3], [1], [80], [0], [8], [0], [1], [6], [4], [0], [5], [0], [0], [1], [0], [0], [1], [0], [1], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [1], [1], [0], [1]]


how can you predict y? You can only predict X. I think y means something else here. Need to stick to f(X)=y ~ model(X)=y

In [28]:
#model.predict(y)

In [29]:
model.summary()
#model.save('/content/gdrive/My Drive/tabular/saved_model') 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features_1 (DenseFeatu multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  3584      
_________________________________________________________________
dense_1 (Dense)              multiple                  4160      
_________________________________________________________________
dense_2 (Dense)              multiple                  65        
Total params: 7,809
Trainable params: 7,809
Non-trainable params: 0
_________________________________________________________________


Nothing needef after this. Some server settings etc. And again same definitions from above.

In [30]:
!saved_model_cli show --dir '/content/gdrive/My Drive/tabular/saved_model/1' --all

In [0]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

In [0]:
!apt-get install tensorflow-model-server


In [0]:
os.environ["MODEL_DIR"] = '/content/gdrive/My Drive/tabular/saved_model/'


In [0]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=saved_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 5 in a separate thread.


In [0]:
!tail server.log


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
URL = '/content/gdrive/My Drive/WA_Fn-UseC_-HR-Employee-Attrition.xlsx'
dataframe = pd.read_excel('/content/gdrive/My Drive/WA_Fn-UseC_-HR-Employee-Attrition.xlsx', sheet_name=0,dtype=dtypes)
dataframe['Attrition']=dataframe.Attrition.eq('Yes').mul(1) # change target from Yes/no to 1/0
dataframe.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [0]:
cont=[]
cat=[]
for key, value in dtypes.items():
    if key!='Attrition':
        if value == "int64":
            cont.append(key)
        else:
            cat.append(key)

In [0]:
print(cat)

['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']


In [0]:
dataframe['BusinessTravel'].unique()

[Travel_Rarely, Travel_Frequently, Non-Travel]
Categories (3, object): [Travel_Rarely, Travel_Frequently, Non-Travel]

In [0]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

940 train examples
236 validation examples
294 test examples


In [0]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Attrition')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
example_batch = next(iter(train_ds))[0]


In [0]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())